<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 colab notebook 主要功能是
- 剪輯 [news-digest](https://news-digest.vercel.app/) 網站當天（或選取日期）的新聞。
- 只能根據新聞分類 (catetories) 剪輯。

執行步驟如下：
1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝用到的模組
2. 根據喜好，填表執行後面的 Jupyter Widgets UI 或 Colab Forms

需要更細的檢索查詢，有如下兩個選擇
- 直接在 [news-digest 網站](https://news-digest.vercel.app/)上搜尋或點選 hash-tags
- 或者點擊 [news_query.ipynb](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb) 執行多條件檢索


### Install 

In [ ]:
import os

fns = ['clip.py', 'op.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
for fn in fns:
    url = f'https://raw.githubusercontent.com/YorkJong/news-digest/main/api/{fn}'
    !wget $url

import clip

#-------------------------------------------------------------------------------

import ipywidgets as widgets

def create_ui():
    fns = sorted(clip.get_recent_journal_filenames(14), reverse=True)
    options = [f[:-3] for f in fns]
    dates = widgets.Dropdown(
        options=options,
        value=options[0],
        description='Date',
        disabled=False,
    )
 
    content = clip.get_latest_journal()
    options = clip.get_categories(content)
    categories = widgets.SelectMultiple(
        options=options,
        #value=[options[0]],
        rows=len(options),
        description='Categories',
        disabled=False,
    )

    with_hashtags = widgets.Checkbox(
        value=True,
        description='show hashtags',
        disabled=False,
    )

    with_headers = widgets.Checkbox(
        value=True,
        description='show headers',
        disabled=False,
    )

    vbox1 = widgets.VBox([dates, categories])
    vbox2 = widgets.VBox([with_hashtags, with_headers])
    output = widgets.Output()
    display(widgets.HBox([vbox1, vbox2]), output)

    def on_date_change(change):
        nonlocal content
        content = clip.get_journal(f'{dates.value}.md')
        categories.options = clip.get_categories(content)

    def on_value_change(change):
        lines = clip.get_lines_of_categories(categories.value, content, with_hashtags.value, with_headers.value)
        with output:
            #print(change['new'])
            output.clear_output()
            print('\n'.join(lines))

    dates.observe(on_date_change, names='value')
    categories.observe(on_value_change, names='value')
    with_hashtags.observe(on_value_change, names='value')
    with_headers.observe(on_value_change, names='value')

### Select categories with Jupyter Widgets
- 按下面那個 code cell 前面的 ► 按鈕，以列出完整的分類(Catetories)選單
- macOS 下操作可按Cmd後用滑鼠複選（Windows 系統則按下Ctrl鍵後以滑鼠複選）
  - 全選可按 Cmd+A (macOS) 或者 Ctrl+A (Windows)

In [6]:
create_ui()

Output()

### Select Categories with the Colab Forms

In [ ]:
#@title Single Selection { run: "auto", display-mode: "form" }
category = "AI" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
show_hashtags = True #@param {type:"boolean"}
show_headers = True #@param {type:"boolean"}

content = clip.get_latest_journal()
lines = clip.get_lines_of_categories([category], content, show_hashtags, show_headers)
text = '\n'.join(lines)
print(text)

In [ ]:
#@title Multiple Selections { run: "auto", display-mode: "form" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
show_hashtags = False #@param {type:"boolean"}
show_headers = False #@param {type:"boolean"}

content = clip.get_latest_journal()
categories = clip.get_categories(content)
categories = clip.get_sublist(categories, first_category, last_category)
lines = clip.get_lines_of_categories(categories, content, show_hashtags, show_headers)
text = '\n'.join(lines)
print(text)


### See Also
- [news_query.ipynb (from GitHub)](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb)